# Project 3 EDA Notebook - Building SQL ETL Pipeline with ML Integrations

In [1]:
!pip install yfinance==0.2.31 --quiet
!pip install jupysql duckdb-engine --quiet
!pip install toml --quiet

You should consider upgrading via the 'c:\users\owner\documents\anaconda package\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'c:\users\owner\documents\anaconda package\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'c:\users\owner\documents\anaconda package\python.exe -m pip install --upgrade pip' command.


In [1]:
import duckdb
import pytz
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta

# %reload_ext sql

C:\Users\owner\Documents\Anaconda Package\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
today = datetime.today()
date = today - timedelta(days=729)
print(f"730 days prior to today's date, {today} is the {date}")

730 days prior to today's date, 2023-10-09 16:26:52.923113 is the 2021-10-10 16:26:52.923113


In [3]:
# Getting historical market data
msft = yf.Ticker("MSFT")
hist = msft.history(period="1d", interval="1h", start=date, end=today)
# hist = msft.history(period="max")
print(hist.shape)
hist

(3502, 7)


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Datetime,,,,,,,
2021-10-11 09:30:00-04:00,292.920013,297.339996,292.744995,297.309998,3955565,0.0,0.0
2021-10-11 10:30:00-04:00,297.309998,297.970001,296.660004,297.019989,2172972,0.0,0.0
2021-10-11 11:30:00-04:00,297.019989,297.390015,295.839996,296.570007,1803150,0.0,0.0
2021-10-11 12:30:00-04:00,296.570007,296.716797,295.616211,295.750000,1763752,0.0,0.0
2021-10-11 13:30:00-04:00,295.760010,296.420013,295.260101,296.126404,1674201,0.0,0.0
...,...,...,...,...,...,...,...
2023-10-09 11:30:00-04:00,326.839996,328.459991,326.829987,327.820007,1732688,0.0,0.0
2023-10-09 12:30:00-04:00,327.825012,328.750000,327.549988,328.750000,1236455,0.0,0.0
2023-10-09 13:30:00-04:00,328.769989,330.299988,328.709991,329.959991,1821354,0.0,0.0


In [4]:
hist_1986_2021 = msft.history(period="max", interval="1d", end=date)
print(hist_1986_2021.shape)
hist_1986_2021

(8968, 7)


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1986-03-13 00:00:00-05:00,0.055004,0.063093,0.055004,0.060396,1031788800,0.0,0.0
1986-03-14 00:00:00-05:00,0.060396,0.063632,0.060396,0.062553,308160000,0.0,0.0
1986-03-17 00:00:00-05:00,0.062553,0.064172,0.062553,0.063632,133171200,0.0,0.0
1986-03-18 00:00:00-05:00,0.063632,0.064172,0.061475,0.062014,67766400,0.0,0.0
1986-03-19 00:00:00-05:00,0.062014,0.062553,0.060396,0.060936,47894400,0.0,0.0
...,...,...,...,...,...,...,...
2021-10-04 00:00:00-04:00,282.285875,282.629653,275.263111,278.072205,31350700,0.0,0.0
2021-10-05 00:00:00-04:00,278.995494,285.232506,278.995494,283.621704,24993000,0.0,0.0
2021-10-06 00:00:00-04:00,280.694763,288.405084,280.429578,287.894318,28002600,0.0,0.0


In [6]:
msft_history = pd.concat([hist_1986_2021, hist])
print(msft_history.shape)
msft_history

(12470, 7)


,Open,High,Low,Close,Volume,Dividends,Stock Splits
1986-03-13 00:00:00-05:00,0.055004,0.063093,0.055004,0.060396,1031788800,0.0,0.0
1986-03-14 00:00:00-05:00,0.060396,0.063632,0.060396,0.062553,308160000,0.0,0.0
1986-03-17 00:00:00-05:00,0.062553,0.064172,0.062553,0.063632,133171200,0.0,0.0
1986-03-18 00:00:00-05:00,0.063632,0.064172,0.061475,0.062014,67766400,0.0,0.0
1986-03-19 00:00:00-05:00,0.062014,0.062553,0.060396,0.060936,47894400,0.0,0.0
...,...,...,...,...,...,...,...
2023-10-09 11:30:00-04:00,326.839996,328.459991,326.829987,327.820007,1732688,0.0,0.0
2023-10-09 12:30:00-04:00,327.825012,328.750000,327.549988,328.750000,1236455,0.0,0.0
2023-10-09 13:30:00-04:00,328.769989,330.299988,328.709991,329.959991,1821354,0.0,0.0
2023-10-09 14:30:00-04:00,329.959991,330.149994,329.058502,329.500000,1882691,0.0,0.0


In [7]:
msft_history = msft_history.rename_axis(["DateTime"]).reset_index().rename(columns={
    "DateTime": "datetime",
     "Open": "open",
     "High": "high",
     "Low": "low",
     "Close": "close",
     "Volume": "volume",
     "Dividends": "dividends",
     "Stock Splits": "stock_splits",
    })

In [8]:
msft_history.tail()

,datetime,open,high,low,close,volume,dividends,stock_splits
12465,2023-10-09 11:30:00-04:00,326.839996,328.459991,326.829987,327.820007,1732688,0.0,0.0
12466,2023-10-09 12:30:00-04:00,327.825012,328.750000,327.549988,328.750000,1236455,0.0,0.0
12467,2023-10-09 13:30:00-04:00,328.769989,330.299988,328.709991,329.959991,1821354,0.0,0.0
12468,2023-10-09 14:30:00-04:00,329.959991,330.149994,329.058502,329.500000,1882691,0.0,0.0
12469,2023-10-09 15:30:00-04:00,329.480011,330.040009,329.209991,329.820007,1678565,0.0,0.0


### Trying SQL Queries

In [9]:
conn = duckdb.connect("stock_data.duckdb", read_only=True)

In [10]:
print(conn.sql("""
SHOW TABLES;
"""))

┌───────────┐
│   name    │
│  varchar  │
├───────────┤
│ msft_data │
└───────────┘



In [11]:
# %%sql

# SELECT *
# FROM msft_data
# LIMIT 5;

print(conn.sql("""
SELECT *
FROM msft_data
LIMIT 5;
"""))

┌─────────────────────┬──────────────────────┬─────────────────────┬───┬────────────┬───────────┬──────────────┐
│      datetime       │      open_price      │      day_high       │ … │   volume   │ dividends │ stock_splits │
│      timestamp      │        double        │       double        │   │   int32    │  double   │    float     │
├─────────────────────┼──────────────────────┼─────────────────────┼───┼────────────┼───────────┼──────────────┤
│ 1986-03-12 21:00:00 │ 0.055004273982196275 │ 0.06309321076953803 │ … │ 1031788800 │       0.0 │          0.0 │
│ 1986-03-13 21:00:00 │ 0.060396426572453206 │  0.0636323702840891 │ … │  308160000 │       0.0 │          0.0 │
│ 1986-03-16 21:00:00 │  0.06255332233106838 │ 0.06417160464127575 │ … │  133171200 │       0.0 │          0.0 │
│ 1986-03-17 21:00:00 │  0.06363240821503023 │  0.0641716291193864 │ … │   67766400 │       0.0 │          0.0 │
│ 1986-03-18 21:00:00 │ 0.062014132940836346 │ 0.06255335391173898 │ … │   47894400 │       0.0 

In [12]:
conn.sql(f"""
CREATE OR REPLACE TABLE msft_data AS
SELECT * FROM {msft_history}
""")

ParserException: Parser Error: syntax error at or near "high"
LINE 3: ...sft_data AS
SELECT * FROM                        datetime        open        high         low  \
                                                  ^